# Exercises

There are three exercises in this notebook:

1. Use the cross-validation method to test the linear regression with different $\alpha$ values, at least three.
2. Implement a SGD method that will train the Lasso regression for 10 epochs.
3. Extend the Fisher's classifier to work with two features. Use the class as the $y$.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import Lasso

## 1. Cross-validation linear regression

You need to change the variable ``alpha`` to be a list of alphas. Next do a loop and finally compare the results.

In [2]:
x = np.array([188, 181, 197, 168, 167, 187, 178, 194, 140, 176, 168, 192, 173, 142, 176]).reshape(-1, 1).reshape(15, 1)
y = np.array([141, 106, 149, 59, 79, 136, 65, 136, 52, 87, 115, 140, 82, 69, 121]).reshape(-1, 1).reshape(15, 1)

x = np.asmatrix(np.c_[np.ones((15, 1)), x])

I = np.identity(2)
alphas = [1.0, 0.1, 0.01, 0.001, 0.0, -0.001, -0.01, -0.1, -1.0]
results = []

for alpha in alphas:
    w = np.linalg.inv(x.T * x + alpha * I) * x.T * y
    w = w.ravel()

    results.append([alpha, w.item(0), w.item(1)])

results = pd.DataFrame(results, columns=['Alpha', 'Bias', 'Coefficient'])
results



,Alpha,Bias,Coefficient
0,1.000,-20.590447,0.710486
1,0.100,-101.723971,1.169788
2,0.010,-167.855340,1.544160
3,0.001,-179.526286,1.610230
4,0.000,-180.924018,1.618142
5,-0.001,-182.343685,1.626179
6,-0.010,-196.199431,1.704617
7,-0.100,-817.017374,5.219094
8,-1.000,26.667097,0.442962


## 2. Implement based on the Ridge regression example, the Lasso regression.

Please implement the SGD method and compare the results with the sklearn Lasso regression results. 

In [3]:
def sgd(X, y, alpha, epochs=10):
    weights = np.zeros(X.shape[1])
    bias = 1

    for _ in range(epochs):
        y_hat = X * weights + bias
        error = y - y_hat
        gradient_weights = -2 * np.sum(X * error) / np.power(np.linalg.norm(X), 2)
        gradient_bias = -(2 / y.size) * np.sum(error)
        weights -= alpha * gradient_weights
        bias -= alpha * gradient_bias

    return bias, weights

In [4]:
x = np.array([188, 181, 197, 168, 167, 187, 178, 194, 140, 176, 168, 192, 173, 142, 176]).reshape(-1, 1).reshape(15, 1)
y = np.array([141, 106, 149, 59, 79, 136, 65, 136, 52, 87, 115, 140, 82, 69, 121]).reshape(-1, 1).reshape(15, 1)

alpha = 0.1
results = []

for i in range(7):
    w = sgd(x, y, alpha, pow(10, i))
    results.append([pow(10, i), w[0], w[1][0]])

lasso = Lasso(alpha=alpha)
lasso_result = lasso.fit(x, y)
results.append(['Sklearn Reference', lasso_result.intercept_[0], lasso_result.coef_[0]])

pd.DataFrame(results, columns=['Epochs', 'Bias', 'Coefficient'])

,Epochs,Bias,Coefficient
0,1,21.293333,0.117653
1,10,50.034916,0.301459
2,100,33.152842,0.401022
3,1000,-82.068370,1.056105
4,10000,-180.880435,1.617895
5,100000,-180.924018,1.618142
6,1000000,-180.924018,1.618142
7,Sklearn Reference,-180.857909,1.617765


## 3. Extend the Fisher's classifier

Please extend the targets of the ``iris_data`` variable and use it as the $y$.

In [5]:
iris_data = load_iris()
iris_df = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris_df.head()
iris_df['target'] = iris_data['target']

x = iris_df[['sepal width (cm)', 'sepal length (cm)']].values
y = iris_df['target'].values.reshape(-1, 1)

dataset_size = np.size(x)

mean_x, mean_y = np.mean(x), np.mean(y)

SS_xy = np.sum(y * x) - dataset_size * mean_y * mean_x
SS_xx = np.sum(x * x) - dataset_size * mean_x * mean_x

a = SS_xy / SS_xx
b = mean_y - a * mean_x

y_pred = a * x + b

pd.DataFrame(y_pred, columns=['Predicted Sepal Width (cm)', 'Predicted Sepal Length (cm)'])

,Predicted Sepal Width (cm),Predicted Sepal Length (cm)
0,0.924785,1.051418
1,0.885212,1.035589
2,0.901042,1.019760
3,0.893127,1.011845
4,0.932700,1.043504
...,...,...
145,0.885212,1.178051
146,0.845640,1.146393
147,0.885212,1.162222
148,0.916871,1.138479
